Anhang
==

In [1]:
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
%matplotlib inline
import numpy as np
from numpy import exp, sqrt, log, pi
from scipy.optimize import curve_fit
from scipy.stats import chi2
from scipy import odr
from scipy.integrate import quad
from scipy.signal import find_peaks


def fehler(name, G, sig_G, G_lit, sig_G_lit):
    print(name)
    print('Relativer Fehler: ', sig_G / G * 100)
    print('Rel. Fehler (Vergleich):', sig_G_lit / G_lit * 100)
    print('Absoluter Fehler: ', G - G_lit)
#    print('Verhältnis:', G / G_lit)
    print('Sigma-Abweichung: ', np.abs(G - G_lit) / sqrt(sig_G ** 2 
                                                         + sig_G_lit ** 2),'\n')
    
    
def fehler_small(name, G, sig_G):
    print(name)
    print('Relativer Fehler: ', sig_G / G * 100)
    

def ergebnis(name, G, sig_G, komma, einheit):
    print(name + ' =', np.round(G, komma), '+/-', np.round(sig_G, komma), einheit)
    

def ergebnis_large(name, G, sig_G, komma, einheit):
    print(name + ' =', np.round(G, komma))
    print('+/-'.rjust(len(name) + 2), np.round(sig_G, komma), einheit)
    

def fitparameter(name, G, sig_G, komma, einheit):
    return name + ' = ' + str(np.round(G, komma)) + '$\pm$' + str(np.round(sig_G, komma)) + einheit


def chi_sq(test_func, x_val, y_val, sig_y_val, popt, number):
    chi2_ = np.sum((test_func(x_val, *popt) - y_val) ** 2 
                   / sig_y_val ** 2)
    dof = len(y_val) - number
    chi2_red = chi2_ / dof

    print("chi2 =", chi2_)
    print("chi2_red =",chi2_red)

    prob = np.round(1 - chi2.cdf(chi2_,dof), 2) * 100 
    print("Wahrscheinlichkeit =", prob, "%\n")

Clément und Desoremes
---

In [2]:
# Messwerte
bh_1 = np.array([58.5, 61.5, 58.7, 58.7, 59.2]) 
bh_3 = np.array([56.5, 56.8, 57.2, 56.6, 56.8])
b = 56
sig_bh = 0.1
h_1 = bh_1 - b
h_3 = bh_3 - b
sig_h = sqrt(2) * sig_bh

kappa_cd = h_1 / (h_1 - h_3)
sig_kappa_cd = kappa_cd * sqrt( (sig_h / h_1) ** 2 + (sqrt(2) * sig_h / (h_1 - h_3)) ** 2)

ergebnis_large('kappa_cd', kappa_cd, sig_kappa_cd, 2, '')

kappa_cd_mean = np.mean(kappa_cd)
sig_kappa_cd_mean = 1 / (sqrt(len(kappa_cd) - 1)) * np.std(kappa_cd)

ergebnis('kappa_cd_mean', kappa_cd_mean, sig_kappa_cd_mean, 2, '')

kappa_cd = [1.25 1.17 1.8  1.29 1.33]
       +/- [0.14 0.06 0.26 0.14 0.13] 
kappa_cd_mean = 1.37 +/- 0.11 


Rüchardt
---

In [11]:
# Messwerte Luft
m_luft = 26.116 # g
sig_m = 0.002
V_luft = 5370 # cm^3
sig_V = 5
r_luft = 15.95 / 2 # mm
sig_r = 0.01
T_luft = 48.31 / 50 # s
sig_T = 0.3 / 50

A_luft = pi * r_luft ** 2 # mm^2
sig_A_luft = 2 * pi * sig_r * r_luft

g = 9.81 # m s^-2

p_luft = 1013 + m_luft * g / A_luft * 10 # hPa
sig_p = sqrt(2 ** 2) # + (sig_m * g / A_luft) ** 2 
             #+ (sig_A_luft * m_luft * g / (A_luft ** 2)) ** 2)# Nur Ablesefehler
    
ergebnis('before: p_luft', p_luft, sig_p, 4, '[hPa]')


kappa_r_luft = 4 * m_luft * V_luft / (r_luft ** 4 * T_luft ** 2 * p_luft) * 10 
sig_kappa_r_luft = kappa_r_luft * sqrt(#(sig_m / m_luft) ** 2 
                                       #+ (sig_V / V_luft) ** 2 
                                       + (4 * sig_r / r_luft) ** 2 
                                       + (2 * sig_T / T_luft) ** 2 
                                       #+ (sig_p / p_luft) ** 2 #Werte zu klein
                                       )

ergebnis('before: kappa_r_luft',kappa_r_luft, sig_kappa_r_luft, 2, '')

sig_kappa_r_luft = 4 * sqrt( (sig_m * V_luft / (r_luft ** 4 * T_luft ** 2 * p_luft)) ** 2 
                            + (m_luft * sig_V / (r_luft ** 4 * T_luft ** 2 * p_luft)) ** 2 
                            + (m_luft * V_luft * 4 * sig_r / (r_luft ** 5 * T_luft ** 2 * p_luft)) ** 2 
                            + (m_luft * V_luft * 2 * sig_T / (r_luft ** 4 * T_luft ** 3 * p_luft)) ** 2 
                            + (m_luft * V_luft  * sig_p / (r_luft ** 4 * T_luft ** 2 * p_luft ** 2)) ** 2) * 10

sig_p = sqrt(2 ** 2 + (10 * sig_m * g / A_luft) ** 2 
             + (10 * sig_A_luft * m_luft * g / (A_luft ** 2)) ** 2)# Ablesefehler + Masse und Fläche

ergebnis('T_luft', T_luft, sig_T, 3, '[s]')
ergebnis('after: p_luft', p_luft, sig_p, 4, '[hPa]')
ergebnis('after: kappa_r_luft',kappa_r_luft, sig_kappa_r_luft, 2, '')

before: p_luft = 1025.8223 +/- 2.0 [hPa]
before: kappa_r_luft = 1.45 +/- 0.02 
T_luft = 0.966 +/- 0.006 [s]
after: p_luft = 1025.8223 +/- 2.0003 [hPa]
after: kappa_r_luft = 1.45 +/- 0.02 


Sigmas
--

In [5]:
# Luft
fehler('kappa_luft', kappa_cd_mean, sig_kappa_cd_mean, kappa_r_luft, sig_kappa_r_luft)

kappa_luft
Relativer Fehler:  8.134982327795857
Rel. Fehler (Vergleich): 1.3394332417173165
Absoluter Fehler:  -0.08029271574347807
Sigma-Abweichung:  0.7108539951508213 



In [6]:
# Argon
kappa_arg_lit = 1.648
fehler('kappa_arg', kappa_r_arg, sig_kappa_r_arg, kappa_arg_lit, 0)

kappa_arg
Relativer Fehler:  1.4068435245022695
Rel. Fehler (Vergleich): 0.0
Absoluter Fehler:  -0.01329448533125599
Sigma-Abweichung:  0.5780776856688609 

